## Importación de Bibliotecas

Comenzamos importando las bibliotecas necesarias para desarrollar la solución utilizando programación genética.

In [1]:
import random
import operator
import copy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score

---
## Carga del Conjunto de Datos

Cargamos el conjunto de datos desde un archivo CSV.

In [3]:
df = pd.read_csv('Datasets/LungCancer.csv')

---
## Exploración y Visualización de los Datos
Para entender mejor nuestros datos, podemos realizar una exploración y visualización inicial.

In [21]:
df.head()

,Gender_Female,Gender_Male,Age_0,Age_1,Age_2,Age_3,Age_4,Age_5,Age_6,Age_7,AnnualSalary_0,AnnualSalary_1,AnnualSalary_2,AnnualSalary_3,AnnualSalary_4,AnnualSalary_5,AnnualSalary_6,AnnualSalary_7,Purchased
0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1
3,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1
4,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1000 entries, 0 to 199
Data columns (total 19 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   Gender_Female   1000 non-null   int64
 1   Gender_Male     1000 non-null   int64
 2   Age_0           1000 non-null   int64
 3   Age_1           1000 non-null   int64
 4   Age_2           1000 non-null   int64
 5   Age_3           1000 non-null   int64
 6   Age_4           1000 non-null   int64
 7   Age_5           1000 non-null   int64
 8   Age_6           1000 non-null   int64
 9   Age_7           1000 non-null   int64
 10  AnnualSalary_0  1000 non-null   int64
 11  AnnualSalary_1  1000 non-null   int64
 12  AnnualSalary_2  1000 non-null   int64
 13  AnnualSalary_3  1000 non-null   int64
 14  AnnualSalary_4  1000 non-null   int64
 15  AnnualSalary_5  1000 non-null   int64
 16  AnnualSalary_6  1000 non-null   int64
 17  AnnualSalary_7  1000 non-null   int64
 18  Purchased       1000 non-null   in

---
## Preprocesamiento de Datos

Definimos una función para convertir los atributos del DataFrame a variables binarias o aplicar one-hot encoding, sin modificar los atributos que ya son binarios

In [23]:
def convert_to_binary(df):
    """
    Convierte los atributos de un DataFrame a binarios o one-hot,
    sin modificar los atributos que ya son binarios.

    Parámetros:
    df : pd.DataFrame
        El DataFrame a convertir.

    Retorna:
    pd.DataFrame
        Un nuevo DataFrame con las columnas convertidas a formato binario o one-hot.
    """
    df_binary = df.copy()

    for column in df_binary.columns:
        # Verifica si la columna es binaria (0/1)
        if df_binary[column].dtype in ['int64', 'float64']:
            if df_binary[column].nunique() == 2:
                df_binary[column].astype(int)  # No modificar columnas binarias
            elif df_binary[column].nunique() > 3:
                # Aplica binning y luego one-hot
                df_binned = pd.cut(df_binary[column], bins=3, labels=False)
                df_one_hot = pd.get_dummies(df_binned, prefix=column)
                df_binary = pd.concat([df_binary, df_one_hot], axis=1).astype(int)
                df_binary.drop(column, axis=1, inplace=True)
            else:
                # Aplica one-hot directamente
                df_one_hot = pd.get_dummies(df_binary[column], prefix=column)
                df_binary = pd.concat([df_binary, df_one_hot], axis=1).astype(int)
                df_binary.drop(column, axis=1, inplace=True)
        elif df_binary[column].dtype == 'object':
            # Si la columna es categórica, aplica one-hot
            df_one_hot = pd.get_dummies(df_binary[column], prefix=column)
            df_binary = pd.concat([df_binary, df_one_hot], axis=1)
            df_binary.drop(column, axis=1, inplace=True)

    return df_binary

In [140]:
df = convert_to_binary(df)
df.columns

Index(['0_b', '0_o', '0_x', '1_b', '1_o', '1_x', '2_b', '2_o', '2_x', '3_b',
       ...
       '39_b', '39_o', '39_x', '40_b', '40_o', '40_x', '41_b', '41_o', '41_x',
       'Class'],
      dtype='object', length=127)

---
## División de Datos en Entrenamiento y Prueba

Dividimos los datos en conjuntos de entrenamiento y prueba para evaluar el rendimiento del modelo

In [4]:
X = df.iloc[:, :-1].values
Y = df.iloc[:, -1].values

Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.2, random_state=10, stratify = Y)

print(len(Xtrain), len(Xtest))

227 57


---
## Definición de los operadores lógicos

Definimos los operadores lógicos que vamos a utilizar para generar los individuos. Los individuos consisten en árboles de decisión formados por operadores y atributos.

In [5]:
OPERATORS = {
    'AND': operator.and_,
    'OR': operator.or_,
    'NOT': lambda x: not x 
}

---
## Clase para representar un nodo del árbol

Creamos una clase para representar un nodo del árbol. La clase contiene métodos para evaluar el nodo de forma recursiva, para hacer copias profundas recurisvamente del nodo actual sin compartir las referencias de sus nodos hijos, representar el nodo como una cadena e imprimir el árbol creado de forma visual.

In [6]:
class Node:
    def __init__(self, value, left=None, right=None, depth = 0):
        self.depth = depth
        self.value = value
        self.left = left
        self.right = right
    def evaluate(self, variables):
        if self.value in OPERATORS: 
            if self.value == 'NOT':
                return OPERATORS[self.value](self.left.evaluate(variables))
            else:
                return OPERATORS[self.value](self.left.evaluate(variables), self.right.evaluate(variables))
        else:
            return variables[self.value]

    def copy(self):
        new_node = Node(self.value, depth=self.depth)

        if self.left:
            new_node.left = self.left.copy()
        if self.right:
            new_node.right = self.right.copy()

        return new_node

    def __str__(self):
        if self.value in OPERATORS:
            if self.value == 'NOT':
                return f"(NOT {self.left})"
            return f"({self.left} {self.value} {self.right})"
        else:
            return str(self.value)

    def print_tree(self, level=0):
        indent = "    " * level  
        print(f" {self.value}")

        if self.left and self.right:
            if self.left:
                print(f"{indent}├──", end="")
                self.left.print_tree(level + 1)
            if self.right:
                print(f"{indent}└──", end="")
                self.right.print_tree(level + 1)

        elif self.left:
            print(f"{indent}└──", end="") 
            self.left.print_tree(level + 1)

        elif self.right:
            print(f"{indent}└──", end="") 
            self.right.print_tree(level + 1)

---
## Función para generar un árbol de operadores aleatorio

Definimos una función que crea un árbol aleatorio con una profundidad definida a partir de los operadores y los Nodos definidos anteriormente.

In [7]:
def generate_random_tree(variables, max_depth=3, actual_depth = 0):
    if max_depth == 0 or (max_depth > 0 and random.random() < 0.3): 
        return Node(random.choice(variables), depth = actual_depth ) 
    else:
        operator = random.choice(list(OPERATORS.keys())) 
        if operator == 'NOT':
            return Node(operator, left=generate_random_tree(variables, max_depth-1, actual_depth+1), depth = actual_depth) 
        else:
            return Node(operator, left=generate_random_tree(variables, max_depth-1, actual_depth+1),
                        right=generate_random_tree(variables, max_depth-1, actual_depth+1), depth = actual_depth)


---
## Función para Transformar los Datos en Diccionarios

Definimos una función  para convertir las filas de las matrices Xtrain y Xtest en listas de diccionarios, mapeando cada valor de la fila a una clave de la forma 'x1', 'x2', etc., para facilitar su uso en el algoritmo.

In [8]:
def generate_mapping(individuo):
    assigment = {}
    for i in range(len(individuo)):
        assigment['x'+str(i+1)] = individuo[i]
    return assigment

XtrainDict = []
for i in range(len(Xtrain)):
    XtrainDict.append(generate_mapping(Xtrain[i]))
XtestDict = []
for i in range(len(Xtest)):
    XtestDict.append(generate_mapping(Xtest[i]))


---
## Función de Generación de Población

Definimos una función para crear una población inicial de árboles aleatorios usando las variables de Xtrain y devuelve esta población junto con las variables para su posterior uso en el algoritmo.

In [9]:
def generate_population(population_size, max_depth = 6):
    population = []
    variables = []
    for i in range(len(Xtrain[1])):
        variables.append('x'+str(i+1))
    for i in range(population_size):
        population.append(generate_random_tree(variables, max_depth=max_depth))
    return population, variables

---
## Funciones de Cálculo de Aptitud

Definimos funciones para calcular la aptitud de la población y de un individuo. La aptitud se basa en la puntuación F1 o exactitud.


In [10]:
def fitness_poblacion(poblacion, umbral = 0.5, metrica = 'f1'):
  predicciones = np.empty((len(Xtrain),len(poblacion)))
  for i in range(len(poblacion)):
    for j in range(len(Xtrain)):
      predicciones[j,i] = poblacion[i].evaluate(XtrainDict[j])
  y_pred = (predicciones >= umbral).astype(int)
  y_true = Ytrain
  scores = np.empty(y_pred.shape[1])

  if metrica == 'f1':
    for i in range(y_pred.shape[1]):
      tp = np.sum((y_true == 1) & (y_pred[:,i] == 1))

      fp = np.sum((y_true == 0) & (y_pred[:,i] == 1))

      fn = np.sum((y_true == 1) & (y_pred[:,i] == 0))

      precision = tp / (tp + fp) if (tp + fp) > 0 else 0
      recall = tp / (tp + fn) if (tp + fn) > 0 else 0

      if precision + recall == 0:
          scores[i] = 0
      else:
        scores[i] = 2 * (precision * recall) / (precision + recall)

  elif metrica == 'acc':
    for i in range(y_pred.shape[1]):
      correct_predictions = np.sum(y_true == y_pred[:,i])

      scores[i] = correct_predictions / len(y_true)

  return scores

def fitness(individuo, umbral = 0.5, metrica = 'f1'):
  probs = np.empty(len(Xtrain))
  for j in range(len(Xtrain)):
      probs[j] = individuo.evaluate(XtrainDict[j])
  y_pred = (probs >= umbral).astype(int)
  y_true = Ytrain
  if metrica == 'f1':
    tp = np.sum((y_true == 1) & (y_pred == 1))

    fp = np.sum((y_true == 0) & (y_pred == 1))

    fn = np.sum((y_true == 1) & (y_pred == 0))

    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0

    if precision + recall == 0:
        return 0.0
    return 2 * (precision * recall) / (precision + recall)

  elif metrica == 'acc':
    return np.sum(y_true == y_pred) / len(y_true)


---
## Función de Selección

Implementamos la selección por torneo para seleccionar individuos para la reproducción.

In [11]:
def seleccion_torneo(fitnessPoblacion, poblacion, k):
    """
    Selección por torneo para algoritmos genéticos.

    :param poblacion: numpy array, la población de soluciones
    :param fitnessPoblacion: numpy array, la evaluación de aptitud de cada solución en la población
    :param k: int, el tamaño del torneo
    :return: numpy array, la población seleccionada después del torneo
    """
    seleccionados = []
    for _ in range(len(poblacion)):
        indices_torneo = np.random.choice(len(poblacion), k, replace=False)
        ganador = indices_torneo[np.argmax(fitnessPoblacion[indices_torneo])]
        seleccionados.append(poblacion[ganador])

    return np.array(seleccionados)

---
## Funciones Auxiliares

A continuación, definimos la profunidad máxima permitida para los árboles y creamos funciones que nos ayudarán en las operaciones de mutación y cruce, estas funciones sirven para calcular la profundidad de un árbol, contar el número de nodos de un árbol o subárbol, contar el número de operadores y operandos y obtener un nodo de forma aleatoria.

In [12]:
MAX_DEPTH = Xtrain.shape[1]//2
def tree_depth(node):
    if node is None:
        return 0
    elif node.left is None and node.right is None:
        return 1
    else:
        left_depth = tree_depth(node.left) if node.left else 0
        right_depth = tree_depth(node.right) if node.right else 0
        return 1 + max(left_depth, right_depth)

def count_nodes(node):
    if node is None:
        return 0
    return 1 + count_nodes(node.left) + count_nodes(node.right)

def contar_operandos_y_operadores(nodo):
    """
    Función recursiva que cuenta cuántos operandos (hojas) y operadores (nodos internos)
    hay en un árbol binario.

    Devuelve una tupla (num_operandos, num_operadores).
    """
    if nodo is None:
        return (0, 0) 

    if not nodo.left and not nodo.right:
        return (1, 0)  

    num_operandos_izq, num_operadores_izq = contar_operandos_y_operadores(nodo.left)
    num_operandos_der, num_operadores_der = contar_operandos_y_operadores(nodo.right)

    num_operadores = 1 + num_operadores_izq + num_operadores_der
    num_operandos = num_operandos_izq + num_operandos_der

    return (num_operandos, num_operadores)

def get_random_node(node, depth=0, max_depth=MAX_DEPTH):
    num_operandos, num_operadores = contar_operandos_y_operadores(node)
    """
    Escoge un nodo aleatorio en el árbol, asegurando que cada nodo tenga una probabilidad similar de ser elegido.
    """
    if depth >= max_depth or (not node.left and not node.right):
        return node

    left_count = count_nodes(node.left) if node.left else 0
    right_count = count_nodes(node.right) if node.right else 0
    total_count = 1 + left_count + right_count 

    choice = random.randint(1, total_count)  
    if choice == 1:
        return node  
    elif choice <= 1 + left_count:
        return get_random_node(node.left, depth+1, max_depth) 
    else:
        return get_random_node(node.right, depth+1, max_depth)

    # depth = tree_depth(node)
    # depth = random.randint(0, depth)
    # for _ in range(depth):
    #     l_o_r = random.randint(0, 1)
    #     if l_o_r == 0 and node.left is not None:
    #         node = node.left
    #     elif node.right is not None:
    #         node = node.right
    #     else:
    #         return node
    # return node

---
## Funciones de Mutación

 Creamos las funciones de mutación diferentes. La primera se encarga de mutar los operadores del árbol, mientras que con la segunda mutamos sus operandos, en ambos casos se sustituyen de forma aleatoria, la función mutation_tree selecciona de forma aleatoria uno de los dos métodos de mutación que acabamos de crear.

In [121]:
def mutate_operator(node, variables):
    if node.value in OPERATORS: 
        possible_operators = list(OPERATORS.keys())
        possible_operators.remove(node.value) 
        to = random.choice(possible_operators)
        
        ## DE NOT -> AND-OR
        if node.value == 'NOT':
            random_variable = random.choice(variables) 
            node_not = Node('NOT', Node(random_variable))

            if to == 'AND':
                # We have to place an 'always True' right tree
                node.right = Node('OR', Node(random_variable), node_not, node.depth)
                
            if to == 'OR':
                # We have to place an 'always False' right tree            
                node.right = Node('AND', Node(random_variable), node_not, node.depth)

        ## DE AND-OR -> NOT 
        if to == 'NOT':
            node.left = Node(node.value, node.left, node.right, node.depth)
            node.right = None
    
        node.value = to

def mutate_operand(node, variables):
    if node.value not in OPERATORS: 
        possible_variables = variables[:]
        possible_variables.remove(node.value) 
        node.value = random.choice(possible_variables) 

def mutate_tree(node, variables, max_depth= MAX_DEPTH):
  if np.random.rand() < 0.1:
    mutation = np.random.choice([0,1])
    node = get_random_node(node).copy()
    if mutation == 0:
      if not node.left and not node.right:
        mutate_operand(node, variables)
      else:
        node = mutate_operator(node)
      return node
    else:
      random = generate_random_tree(variables, max_depth=max_depth)
      node.value = random.value
      node.left = random.left
      node.right = random.right

In [122]:
random.seed(64)
poblacion, variables = generate_population(1, 3)
individuo = poblacion[0]
individuo.print_tree()
print("---------------------------")
mutate_operator(individuo, variables)
individuo.print_tree()
print("---------------------------")

 NOT
└── NOT
    └── AND
        ├── x7
        └── x9
---------------------------
 AND
├── NOT
    └── AND
        ├── x7
        └── x9
└── OR
    ├── x5
    └── NOT
        └── x5
---------------------------


In [108]:
random.seed(10)
poblacion, variables = generate_population(1, 3)
individuo = poblacion[0]
individuo.print_tree()
print("---------------------------")
mutate_operator(individuo, variables)
individuo.print_tree()
print("---------------------------")

 OR
├── AND
    ├── x16
    └── NOT
        └── x6
└── x16
---------------------------
 NOT
└── OR
    ├── AND
        ├── x16
        └── NOT
            └── x6
    └── x16
---------------------------


---
## Función de Cruce

Implementamos el cruce entre dos árboles, estos intercambian subárboles aleatorios creando así dos hijos, asegurándose de que no se excede la profundidad máxima definida.

In [133]:

def crossover(tree1, tree2, max_depth=MAX_DEPTH):
    subtree1 = get_random_node(tree1, max_depth=max_depth)
    subtree2 = get_random_node(tree2, max_depth=max_depth)

    depth_subtree1 = tree_depth(subtree1)
    depth_subtree2 = tree_depth(subtree2)

    if not(depth_subtree1 + tree_depth(tree2) <= max_depth and depth_subtree2 + tree_depth(tree1) <= max_depth):
      count = 0
      while(depth_subtree1 + tree_depth(tree2) <= max_depth and depth_subtree2 + tree_depth(tree1) <= max_depth and count < 10):
          subtree1 = get_random_node(tree1, max_depth=max_depth)
          subtree2 = get_random_node(tree2, max_depth=max_depth)
          count += 1
    
    subtree1.value, subtree2.value = subtree2.value, subtree1.value
    subtree1.left, subtree2.left = subtree2.left, subtree1.left
    subtree1.right, subtree2.right = subtree2.right, subtree1.right
    
    return tree1, tree2

---
## Comprobación de Cruce

Con este código nos aseguramos de que los cruces se realizan como es debido, de forma visual.

In [36]:
random.seed(10)
poblacion, variables = generate_population(2, 3)
poblacion[0].print_tree()
print("---------------------------")
poblacion[1].print_tree()
print("---------------------------")
hijo1, hijo2 = crossover(poblacion[0].copy(), poblacion[1].copy(), max_depth= 6)
hijo1.print_tree()
print("---------------------------")
hijo2.print_tree()

 OR
├── AND
    ├── x16
    └── NOT
        └── x6
└── x16
---------------------------
 AND
├── NOT
    └── AND
        ├── x14
        └── x5
└── OR
    ├── NOT
        └── x9
    └── NOT
        └── x10
---------------------------
 OR
├── AND
    ├── x16
    └── OR
        ├── NOT
            └── x9
        └── NOT
            └── x10
└── x16
---------------------------
 AND
├── NOT
    └── AND
        ├── x14
        └── x5
└── NOT
    └── x6


---
## Funciones de Evaluación

Se define la función para evaluar el modelo en el conjunto de test, usando la métrica f1 por defecto pero también con la opción de usar la métrica de exactitud.

In [134]:
def evaluate(goat, umbral = 0.5, metrica = 'f1'):
  probs = np.empty(len(Xtest))
  for j in range(len(Xtest)):
      probs[j] = goat.evaluate(XtestDict[j])
  y_pred = (probs >= umbral).astype(int)
  y_true = Ytest
  if metrica == 'f1':
    tp = np.sum((y_true == 1) & (y_pred == 1))

    fp = np.sum((y_true == 0) & (y_pred == 1))

    fn = np.sum((y_true == 1) & (y_pred == 0))

    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0

    if precision + recall == 0:
        return 0.0
    return 2 * (precision * recall) / (precision + recall)

  elif metrica == 'acc':
    return np.sum(y_true == y_pred) / len(y_true)


---
## Función Main

Definimos la función main que ejecuta el algoritmo genético. Almacena el valor del mejor individuo y evalúa su fitnes y su F1 en test. Además se devuelve el dibujo del mejor individuo junto con su expresión. 

In [137]:
def main():
  num_generaciones = 120
  
  # Cada cuantas generaciones aumentar el tamaño del torneo 
  presion_selectiva = 20
  
  iter_por_fase = num_generaciones // 20
  gen_actual = 0
  
  profundidad_init = Xtrain.shape[1]//2
  poblacion, variables = generate_population(10, profundidad_init)
  goat = generate_random_tree(variables, max_depth=1)
  
  elitismo =  True
  fitnessPob = 0
  max_fit = 0
  rng = np.random.default_rng()
  while (num_generaciones > gen_actual and np.max(fitnessPob) < 1.0):

    fitnessPob =  fitness_poblacion(poblacion, umbral = 0.5, metrica = 'f1')
    if max_fit < np.max(fitnessPob):
        goat = poblacion[np.argmax(fitnessPob)].copy()
        max_fit = np.max(fitnessPob)

    seleccionados = seleccion_torneo(fitnessPob, poblacion, 2 + (gen_actual // presion_selectiva))

    hijos = []
    indices = np.arange(len(seleccionados))
    rng.shuffle(indices)

    for i in range(0, len(seleccionados), 2):
      padre1, padre2 = indices[i], indices[i+1]
      hijo1, hijo2 = crossover(seleccionados[padre1].copy(), seleccionados[padre2].copy(), max_depth= profundidad_init
                              + (gen_actual //  iter_por_fase))
      hijos.append(hijo1)
      hijos.append(hijo2)

    for hijo in hijos:
      mutate_tree(hijo, variables)

    if elitismo:
      hijos.pop()
      hijos.append(poblacion[np.argmax(fitnessPob)])

    poblacion = np.array(hijos)

    if gen_actual % 5 == 0:
      print("Generación {} Mejor fitness de la generación {}".format(gen_actual, np.max(fitnessPob)))
    gen_actual += 1
  return goat 

Generación 0 Mejor fitness de la generación 0.7952477023088993
Generación 5 Mejor fitness de la generación 0.7952477023088993
Generación 10 Mejor fitness de la generación 0.7962586232369956
Generación 15 Mejor fitness de la generación 0.7962588358297157
Generación 20 Mejor fitness de la generación 0.7962588358297157
Generación 25 Mejor fitness de la generación 0.7962588358297157
Generación 30 Mejor fitness de la generación 0.7962588358297157
Generación 35 Mejor fitness de la generación 0.7962588358297157
Generación 40 Mejor fitness de la generación 0.7962588358297157


KeyboardInterrupt: 

In [21]:
if __name__ == "__main__":
    goat = main()

Generación 0 Mejor fitness de la generación 0.6299559471365639
Generación 5 Mejor fitness de la generación 0.8370044052863436
Generación 10 Mejor fitness de la generación 0.9074889867841409
Generación 15 Mejor fitness de la generación 0.9911894273127754
Generación 20 Mejor fitness de la generación 0.9911894273127754
Generación 25 Mejor fitness de la generación 0.9911894273127754
Generación 30 Mejor fitness de la generación 0.9911894273127754
Generación 35 Mejor fitness de la generación 0.9911894273127754
Generación 40 Mejor fitness de la generación 0.9911894273127754
Generación 45 Mejor fitness de la generación 0.9911894273127754
Generación 50 Mejor fitness de la generación 0.9911894273127754
Generación 55 Mejor fitness de la generación 0.9911894273127754
Generación 60 Mejor fitness de la generación 0.9911894273127754
Generación 65 Mejor fitness de la generación 0.9911894273127754
Generación 70 Mejor fitness de la generación 0.9911894273127754
Generación 75 Mejor fitness de la generaci

In [22]:
print('Fitnes del mejor individuo:', fitness(goat, metrica = 'f1'))
print('Aptitud en el conjunto de prueba:', evaluate(goat, umbral = 0.5, metrica = 'f1'))

Fitnes del mejor individuo: 0.9876543209876543
Aptitud en el conjunto de prueba: 0.975609756097561


In [23]:
goat.print_tree()
print(goat)

 AND
├── OR
    ├── OR
        ├── x11
        └── NOT
            └── x11
    └── AND
        ├── x7
        └── x14
└── NOT
    └── AND
        ├── x18
        └── x18
(((x11 OR (NOT x11)) OR (x7 AND x14)) AND (NOT (x18 AND x18)))
